#Libraries and Data

In [1]:
#import libraries
import numpy as np
import pandas as pd
import itertools
from prophet import Prophet
import yfinance
import pickle
import xgboost as xgb
from datetime import datetime
from ta import add_all_ta_features

from dataProcessing import DataProcessing
# Remove Future Warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import StandardScaler




from sklearn.model_selection import train_test_split

# Machine Learning
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

# Evaluation
from sklearn.metrics import precision_score

# Reporting
import matplotlib.pyplot as plt

/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
def load_and_setup_data(sybmol,input_data):
    df = pd.read_csv("../stock_historical_data/{}.csv".format(sybmol))
  #  df.set_index("Date", inplace=True)
    new_data = pd.DataFrame(input_data)
    new_data.set_index("Date", inplace=True)
    #concatenated_df = pd.concat([df,new_data])
    return df

def fetch_stock_data(tickers):
    data = yfinance.download(tickers,period='1d')
    return data
    

In [3]:
symbols = ["CONCOR.NS","ELGIEQUIP.NS"]
#symbols = ["OLECTRA.NS","CONCOR.NS","ELGIEQUIP.NS","IOC.NS","BEL.NS","TATAELXSI.NS","^NSEI"]


Icdate = 0
Iresult = 1
IisClassification = 2
Imodel = 3
Istock = 4
Iclose = 5
Iprob = 6
  # Add the tickers you want to fetch data for
loaded_models = {}
result = []

final_result = {}
final_target = {}
stock_data = fetch_stock_data(symbols)
#print(stock_data)
finaldata = {}
current_date = datetime.now().strftime("%d-%m-%Y")

for symbol in symbols:
    stock_name = symbol
    if symbol == "^NSEI":
        stock_name = "NSEI"
 
    input_data = {
                    # "Open" : stock_data.head(1).Open[ticker].values[0],
                    "Open" : [stock_data.head(1).Open[symbol].values[0]],
                    "Close" : [stock_data.head(1).Close[symbol].values[0]],
                    "High" : [stock_data.head(1).High[symbol].values[0]],
                    "Low" : [stock_data.head(1).Low[symbol].values[0]], 
                    "Volume" : [stock_data.head(1).Volume[symbol].values[0]],
                    "Date":[np.datetime_as_string(stock_data.index, unit='D')[0]]
    }
    #print(input_data)
    data = load_and_setup_data(symbol,input_data)
    data = data[["Open", "High", "Low", "Close","Volume"]]

   # data.to_csv(f"../stock_historical_data/{symbol}.csv")
    data = add_all_ta_features(data, open="Open", high="High", low="Low", close="Close", volume="Volume", fillna=True)
    # Find NaN Rows
    #na_list = data.columns[data.isna().any().tolist()]
   # data.drop(columns=na_list, inplace=True)
    # Handle inf values
    data.replace([np.inf, -np.inf], np.nan, inplace=True)
    data.dropna(axis=1, inplace=True)
    #data = data.drop(['volatility_kchi','volatility_kcli'], axis=1)

    #df_stationary.head()
   # print(data)
    non_stationaries = []
    for col in data.columns:
        if col != "volatility_kchi" and col != "volatility_kcli":
            dftest = adfuller(data[col].values)
            p_value = dftest[1]
            t_test = dftest[0] < dftest[4]["1%"]
            if p_value > 0.05 or not t_test:
                non_stationaries.append(col)
    
    df_stationary = data.copy()
    
 
  
    loaded_model = {}
    feature_item = {}
    with open('../TrainedModel/indicator/{}_model_2.pkl'.format(symbol), 'rb') as f:
        loaded_model = pickle.load(f)
    with open('../TrainedModel/indicator/{}_features.txt'.format(symbol), 'rb') as f:
        feature_item = pickle.load(f)
   # print(feature_item,symbol,len(feature_item))
    #print("point zero :" ,len(df_stationary.columns))

    df_stationary[non_stationaries] = df_stationary[non_stationaries].pct_change()
    df_stationary = df_stationary.iloc[1:]
    # Find NaN Rows
    na_list = df_stationary.columns[df_stationary.isna().any().tolist()]
    df_stationary.drop(columns=na_list, inplace=True)
    
    #print("point one :" ,len(df_stationary.columns))
    df_stationary = df_stationary[feature_item]
    #print("point two :" ,len(df_stationary.columns))

#    print(df_stationary)
    df_stationary = df_stationary.iloc[1:]
    
    df_stationary[np.isinf(df_stationary)] = np.nan  # Replace inf with NaN
    df_stationary = df_stationary.dropna()
    print("point three",len(df_stationary.columns),symbol)
    has_inf = "no inf data"
    if np.isinf(df_stationary).any().any():
        has_inf = "has inf data"
    df_stationary.replace(np.inf, np.nan, inplace=True)
    df_stationary.dropna(inplace=True)

        
    #print("check infifty value in",has_inf)
   # print(symbol," :",len(df_stationary.columns),len(feature_item))
    finaldata[symbol] = df_stationary 
    df_stationary = StandardScaler().fit_transform(df_stationary)
    y_pred = loaded_model.predict(df_stationary)
    y_pred = loaded_model.predict(df_stationary)

    #print("got result",symbol)
    # Specify Target

    #final training
    loaded_models[symbol] = loaded_model
    stock_name_only = stock_name.replace(".NS","")
   
    train_yhat = loaded_model.predict(df_stationary[-1:])
    train_yhat_proba = loaded_model.predict_proba(df_stationary[-1:])
    greater = 0
    one = train_yhat_proba[0][0]
    two = train_yhat_proba[0][1]
    if one > two:
        greater = one*100
    else:
        greater = one*100
    print(greater)
    final_result[symbol] = [current_date,train_yhat[0],1,'indicator',stock_name_only,stock_data.head(1).Close[symbol].values[0],round(greater,2)]
    result.append([current_date,train_yhat[0],1,'indicator',stock_name_only,stock_data.head(1).Close[symbol].values[0],round(greater,2)])

print(result)


[*********************100%***********************]  2 of 2 completed


/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)


point three 14 CONCOR.NS
20.222552902824965


/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)


point three 14 ELGIEQUIP.NS
37.77146053486274
[['08-07-2023', 1, 1, 'indicator', 'CONCOR', 670.6500244140625, 20.22], ['08-07-2023', 1, 1, 'indicator', 'ELGIEQUIP', 538.7000122070312, 37.77]]


In [4]:

dataa = { "data":[]} 

for res in result:
    dataa["data"].append({ 
            "cdate": res[Icdate],
            "result": str(res[Iresult]),
            "pre_close" : str(res[Iclose]),            
            "isClassification": res[IisClassification],
            "model": res[Imodel],
            "stock": res[Istock],
            "prob": str(res[Iprob]),
    })
    
dataa

{'data': [{'cdate': '08-07-2023',
   'result': '1',
   'pre_close': '670.6500244140625',
   'isClassification': 1,
   'model': 'indicator',
   'stock': 'CONCOR',
   'prob': '20.22'},
  {'cdate': '08-07-2023',
   'result': '1',
   'pre_close': '538.7000122070312',
   'isClassification': 1,
   'model': 'indicator',
   'stock': 'ELGIEQUIP',
   'prob': '37.77'}]}

In [35]:
import requests
import json

url = "https://website-development-kerala.com/api_214124524/ai_model_daily_runner.php"

response = requests.post(url, json=dataa)
print(response.status_code)
print(response)

200
<Response [200]>


In [36]:
#final_target

{}

In [11]:
for symbol in symbols:
  
    # Set Target (for Supervised ML later on)
    finaldata[symbol]["TARGET"] = -1
    finaldata[symbol].loc[finaldata[symbol]["Close"].shift(-1) > finaldata[symbol]["Close"], "TARGET"] = 1
    finaldata[symbol].dropna(inplace=True)
    #print(finaldata)
    df_tar = finaldata[symbol][["TARGET"]]
    df_stationar = StandardScaler().fit_transform(finaldata[symbol].iloc[:, :-1])
   # print(df_tar)

    retrained_model = loaded_models[symbol].fit(df_stationar,df_tar)
    print(df_stationar)
    with open('../TrainedModel/indicator/{}_model_2.pkl'.format(symbol), 'wb') as f:
        pickle.dump(retrained_model, f)


/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


[[-0.31428264 -0.02744056  0.09215209 ... -0.02744056 -0.01738112
  -0.01709641]
 [-0.56167828 -0.36886506 -0.15656718 ... -0.36886506 -0.44388428
   0.68235988]
 [-0.4368701  -0.5032231   0.01959147 ... -0.5032231  -0.61329172
   0.44787221]
 ...
 [ 0.06939993  0.29835383  0.01982563 ...  0.29835383  0.38437113
  -0.01067312]
 [ 0.43331612  0.62427664  0.01974788 ...  0.62427664  0.78130371
  -0.00131855]
 [-0.20441999 -0.75016398  0.02033948 ... -0.75016398 -0.92700636
  -0.04076564]]


/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


[[-6.21295226e-01 -1.68842650e+00  5.11592643e+01 ... -1.30402938e+00
  -1.68842650e+00 -1.79636167e+00]
 [-1.30826745e+00 -1.70721296e-01  4.51737066e+00 ... -1.30402938e+00
  -1.70721296e-01 -1.63778151e-01]
 [-4.16647424e-02 -2.35723593e-01  1.31675944e+00 ... -1.30402938e+00
  -2.35723593e-01 -2.32107777e-01]
 ...
 [ 1.39595337e-01  5.36911872e-02 -5.64312258e-03 ... -3.88517055e-01
   5.36911872e-02  7.10654712e-02]
 [-2.62791150e-01 -1.04993518e-01 -2.07362000e-02 ... -6.65080619e-01
  -1.04993518e-01 -9.48260821e-02]
 [-5.26709843e-02 -9.31367179e-02 -2.22743846e-02 ... -9.00761033e-01
  -9.31367179e-02 -8.24026061e-02]]
